In [4]:
%load_ext autoreload
%autoreload 2



In [2]:
orbit = HW2AV(S2F('6,5,3/2,4,5/2'), rtype='B',report=1)
orbit = HW2AV(S2F('3.1,2.3,1.1,-4,-4.1,2.5,1.9,2,2.1,0'),rtype = 'C', report = 1)


Type B_5:
 AV_C(L(6,5,3/2,4,5/2)) 
  = (3, 3, 1, 1, 1, 1, 1) 
 AVC-dim= 30
***
***
*
*
*
*
*
Type C_10:
 AV_C(L(31/10,23/10,11/10,-4,-41/10,5/2,19/10,2,21/10,0)) 
  = (14, 4, 2) 
 AVC-dim= 192
**************
****
**


In [3]:
orbit = HW2AV(S2F('2,3,4,5,1'),rtype = 'D', report = 1)
orbit = HW2AV(S2F('2,3,4,5,1,1/2,3/2,2.3'),rtype = 'D', report = 1)

Type D_5:
 AV_C(L(2,3,4,5,1)) 
  = (4, 4, 1, 1) 
 AVC-dim= 32
****
****
*
*
Type D_8:
 AV_C(L(2,3,4,5,1,1/2,3/2,23/10)) 
  = (7, 7, 1, 1) 
 AVC-dim= 102
*******
*******
*
*
